## Workday Data Issue Semi-automation Tool (for action U only)

This is a semi-automation tool which to facilate to performa the manual data update for workday data issues.   
  
**Attention:** 
1. This tool only handles the action type for `U`    
1. `D` will be handled by another notebook   
1. For action `US`, `DS` or `R`, it is out of scope. We will not use tool to handle them. 



+ Step 1: **Check** if there's any open entries in the box folder. 
https://ibm.ent.box.com/file/731566115825

+ Step 2: **Validation** for cnums and field names
+ _Step 3: if there's open ticket in box file, we will **create the jira ticket** to handle._  
https://jsw.ibm.com/browse/ODMODMR-91

+ _Step 4: **Update the box file** spreadsheet with correct jira ticket number, and check the status from "Not Started" to "in processing" _

+ Step 5: Read the spreadsheet and **generate the contents** for jira task (for action = 'U' and status = 'in processing')

+ Step 6: Prepare the **SQL** statement

+ Step 7: prepare the **JCL** and write to local file system

+ Step 8: Copy the JCL to ODMLD.PRD.RUN(**ODMSUBRJ**)

+ Step 9: Trigger the **ODMP911** OPC application **(out of tool scope)**

+ Step 10: Download the **dry run result** from the MF server

+ _Step 11: post the dry run result onto jira task, leave comments and **request BPO for approval **_

+ _Step 12: Once approval from BPO is received, please uploade the JCL again, and perform **real run** (rerun step 5, 6, 7, 8)_

+ _Step 13: Download the **real run result** from server to you local machine and then post into jira task, and **close the jira task**_

+ _Step 14: Go to BOX file, populate the close date and change the status for those ticket to be **completed (out of tool scope)**_

In [ ]:
# Parameters
user = 'C943511'  # the userid in the JCL ODMSUBRJ, can be changed to Hans's ID e.g.
#user = 'NL62958'  # the userid in the JCL ODMSUBRJ, can be changed to Hans's ID e.g.
file_id = '731566115825'  # the box file id which holds the request spreadsheet
tbname = 'ODMT_EMPLOYEE'  # you can use the odm developer view if you don't have the access to E01
#tbname = 'ODMH_EMPLOYEE'  # you can use the odm developer view if you don't have the access to E01
action = 'U'
#file_id = '761862313652'  # for test purpose

In [ ]:
import sys
sys.path.append('/odm_modules')
from common_func import odm_conn
sys.path.append('/app')
#from BOX import box_oauth as box
from BOX import box_jwt as box
import pandas as pd
import numpy as np
import datetime
def cell_format(v):                                                             
    return str(v) if isinstance(v, (int)) else str(int(v)) if isinstance(v,float) else str(v.date()) if isinstance(v, datetime.date) else v.decode('utf-8') if isinstance(v, bytes) else '' if v is None else v.strip()                 

### Step 1 Read the file from box folder to check if there's any open items. 

+ If there's open entries, then show the open entries and do some pre-validation before processing
+ if there's no open entries, then STOP here. 

In [ ]:
client = box.get_box_client()
xlsx_file = client.file(file_id).get().name
print('File name for the data change request is:\n\t{}'.format(xlsx_file))
xlsx_content = client.file(file_id).content()
df = pd.read_excel(xlsx_content, sheet_name = 'Manual Action request', header =1 ).fillna('').applymap(cell_format)
df = df.loc[df.RCNUM != '',:]
df.RCNUM = df.RCNUM.str.upper()
request_df = df.loc[(df.status.isin(['Not Started', ''])) & (df.action == action)]
print('there are {} pending requests from Workday team'.format(request_df.shape[0]))
request_df

### Step 2 Validataion (only validate those ticket which are in status 'not started')

we perform the validation before raise the jira task  
+ step 2.1 CNUM validation
+ step 2.2 Field name validation. This tool can only handle the data change request for E01 table. (if some fields are from other E table, please update the action into 'US' and handle the request purely manually)
+ step 2.3 For update request, check if there's any duplicate request which are on the same cnum and with different new value. 
+ _(future improvement) step 2.4: value validation. Validate the value given in the sheet again the data type of the ODM field_. 



In [ ]:
# step 2.1 Cnum validation to check if the cnum could be found in ODM
cnums_from_file = set(request_df.RCNUM.unique())
cnum_list = ','.join(["'{}'".format(cnum.strip()) for cnum in cnums_from_file])
if len(cnums_from_file) !=0: 
    sql = 'SELECT RCNUM, DUPDATE, CACTIVE FROM ODMPRD.{} WHERE RCNUM IN ({})  '.format(tbname, cnum_list)
    #print(sql)
    with odm_conn.odm_adhoc('prod') as odmprd_adhoc:
        result = odmprd_adhoc(sql)
    result_df = pd.DataFrame(result)
    cnums_result = set(result_df.RCNUM.str.strip())
    diff = cnums_from_file - cnums_result
    if len(diff) != 0: 
        print('ERROR!!!!: The following cnum could not be found in ODM database, please request WD COE team to check:\n {}'.format(cnums_from_file - cnums_result))
    else: 
        print('CNUM VALIDATION PASSED!')
        print('IMPACTED {} CNUMS :'.format(len(cnums_result)))
        for cnum in cnums_result: 
            print('\t{}'.format(cnum))

else:
    print('no pending requests in the spreadsheet!')

In [ ]:
# step 2.2
# for those df.action == 'U', we need all those field_to_be_updated fields to be the field name in E01 table
if action == 'U':
    fields = request_df.field_to_be_updated.str.strip().unique()
    sql = "select ctid, ccolname, ccolfmt, ccolname from odmprd.odmt_ddict_columns where substr(ctid,1,1) = 'E' " # rz2 table 
    with odm_conn.odm_adhoc('prod') as odmprd_adhoc:
        result_df = pd.DataFrame(odmprd_adhoc(sql))
    e01_fields = result_df.loc[result_df.CTID == 'E01', 'CCOLNAME'].str.strip()
    if (set(fields) <= set(e01_fields)):
        print('FIELDS VALIDATION PASSED!\n\nThe following fields in E01 table will be updated')
        for field in fields: 
            print('\t {}'.format(field))

    else:
        print('''ERROR: FIELDS VALIDATION FAILED! 
        The following field(s) {} could not be found in E01 table.
        
        Note: If those fields are from table other than E01 table, please update the action 
        from U for US (Updata Subtype)
        Those fields has to be handle purely manually and can not make use of this tool currently.
        If workday COE team made a mistake on the fields name, please request the team to provide 
        a valid fields name'''.format(set(fields) - set(e01_fields)))
else: 
    print('NO REQUEST FOR DATA UPDATE, THIS STEP SKIPPED!')

In [ ]:
# step 2.3, get the cnum list and field list from the dataframe and do a simple varification
if action == 'U':
    update_df = request_df.loc[:, ['RCNUM', 'action', 'field_to_be_updated', 'old_value', 'new_value']].drop_duplicates()
    temp_df = update_df.groupby(['RCNUM', 'field_to_be_updated']).size()
    if any(temp_df >=2) :
        temp_df = temp_df.loc[temp_df >=2]
        print(temp_df)
        print( 'ERROR: CONFLICT FOUND! Please contact with WD COE team to fix the issue...')
    else:
        print( '\n\nPASSED! NO CONFLICT FOUND...')

update_df

### Step 3:  Create the JIRA task to address data change request (Manual)

if there's open ticket in box file, we will create the jira ticket to handle.   
Make sure the jira task is under the following epic   
https://jsw.ibm.com/browse/ODMODMR-91

**Please create separate jira task for U and D respectively**


### Step 4: Update the box file spreadsheet (Manual Step)  
+ Put the jira task number in the spreadsheet  
https://ibm.ent.box.com/file/731566115825
+ Put your name in the spreadsheet
+ Change the status from "Not Started" to "In processing"

### Step 5: Read the spreadsheet and generate the contents for jira task (for action = 'U' and status = 'in processing') 

In [ ]:
# step 5.1 
# read all the ticket for update requests
client = box.get_box_client()
xlsx_file = client.file(file_id).get().name
xlsx_content = client.file(file_id).content()
df = pd.read_excel(xlsx_content, sheet_name = 'Manual Action request', header =1 ).fillna('').applymap(cell_format)
df = df.loc[df.RCNUM != '',:]
df.RCNUM = df.RCNUM.str.upper()
request_df = df.loc[(df.status == 'In Processing') & (df.action == action)]
print('there are {} pending {} requests from Workday team'.format(request_df.shape[0], action))
# get the Category information and task number
cat_df = pd.read_excel(xlsx_content, sheet_name = 1, header = 1, dtype = str).fillna('')
jira_nbrs = request_df.RTC_task_number.unique()
#rtc_nums = ['RA' + rtc_num.split('-')[1].zfill(5) for rtc_num in rtc_nums]
#rtc_num = rtc_nums[0]
#print('\nthe jira number is {}'.format(rtc_num))
request_df

In [ ]:
print('there are {} jira ticket(s) for {} action'.format(len(jira_nbrs), action))
for jira_nbr in jira_nbrs: 
    print('https://jsw.ibm.com/browse/{}'.format(jira_nbr))
if len(jira_nbrs) >= 2: 
    while True: 
        print('please tell which ticket to be handled this time? ({})'.format(', '.join(jira_nbrs)))
        jira_nbr = input()
        if jira_nbr in jira_nbrs:
            print('{} will be handled this time'.format(jira_nbr))
            break
        else: 
            print('{} is not a jira ticket number in status "in-processing", please try again'.format(jira_nbr))
if len(jira_nbrs) == 1: 
    jira_nbr = jira_nbrs[0]

rtc_num = 'RA' + jira_nbr.split('-')[1].zfill(5) 
request_df_current = request_df.loc[request_df.RTC_task_number == jira_nbr]
request_df_current
    

#### STEP 5.2 Copy the contents into jira task
also download the box spreadsheet and upload the spreadsheet to jira task 


In [ ]:
# step 5.2 
# get the zen desk ticket number (get the information form the dataframe.)
print( "\n\n[ZEN desk tickets: ]")
print( '\n'.join(request_df_current.COE_zendesk_number.unique()))

# step 5.3
# get the category number, then use the sheet 2 -- category description sheet to get the description of each category
cat_dict = cat_df.loc[:, ['cat', 'description']].set_index('cat')['description'].to_dict()
cat_list = request_df_current.issue_category.unique()
cnum_list = list(request_df_current.RCNUM.unique())
for cat in cat_list:
    print("\n[{}]: \n {}".format(cat, cat_dict[cat]))
print( '\n[List of cnum to be updated] (impacted {} cnums in total)'.format(len(cnum_list)))


# step 5.4 , query the odm database for those cnums and print( out the query result)
sql_step2 = ''
for cnum in cnum_list:
    print( cnum + ':')
    condition = request_df_current.RCNUM == cnum
    update_item = []
    for index, row in request_df_current.loc[condition, ['field_to_be_updated', 'new_value']].iterrows():
        print( '\t Field Name: {:10s}New Value: {:20s}'.format(*row))
        update_item.append("{} = '{}'".format(*row))
    sql_step2 += "\nUPDATE ODMPRD.ODMT_EMPLOYEE SET\n   {} \n WHERE RCNUM = '{}';\n".format(',\n   '.join(update_item), cnum)

### STEP 6 Prepare the SQL statement 

In [ ]:
# step 6 prepare the SQL statements
distinct_fd_list = request_df_current.field_to_be_updated.unique()
cnum_list_with_quote = [ "'{}'".format(cnum)    for cnum in cnum_list]
sql_step1 = '''
SELECT RCNUM,
{}
FROM ODMPRD.ODMT_EMPLOYEE
WHERE RCNUM IN
({}) ; '''.format(', \n'.join(distinct_fd_list), ', \n'.join(cnum_list_with_quote))

# step 7 prepare the sql
sql = '''
-- step 1
 {0}

-- step 2
 {1}

-- step 3
 {0}

-- step 4
 ROLLBACK;

-- step 5
 {0}
 '''.format(sql_step1, sql_step2)

print( sql)

### STEP 7 prepare the JCL and write to local file system

In [ ]:
result_file_name = '{0}.T{1}.RESULT.TXT'.format(user, rtc_num)
print(f'result file name is : \n{result_file_name}')

In [ ]:
jcl_template = '''//ODMSUBRJ JOB ('QQY5ODM,39U28,SU45RMR,STF-08'),'ODM team support',             
//            MSGCLASS=P,MSGLEVEL=(1,1),REGION=4096K,USER=PCODM,                
//            NOTIFY={0}                                                    
//JOBOUT OUTPUT DEFAULT=YES,JESDS=ALL,FORMS=ODM1                                
//LIBS     JCLLIB  ORDER=(ISCTL.PRODPROC.ODM)                                   
//INCLMEM INCLUDE MEMBER=ODMNA        level dependent JCL variables             
//**********************************************************************        
//* Name of member  : ODMSUBRJ                                                  
//* Function        : This triggers the OPC Application called ODMP911          
//*                   The job running is ODM911SJ, which starts the             
//*                   required process to run, this ODMSUBRJ job                
//*                                                                             
//* ODM Team instructions                                                       
//*                   Always copy the STEP you want to run just below           
//*                   these instructions. Always leave the other steps          
//*                   in and ensure 2 there is the // above AND that            
//*                   there is NO JOB CARD                                      
//*                   When "You're done" put the // below this comments         
//*                                                                             
//* Restart Info    : Fix the problem and restart at the step that              
//*                   failed.                                                   
//* Operators Instructions                                                      
//*                 : NONE                                                      
//**********************************************************************        
//*---------------------------------------------------------------------        
//* This job is for the RTC story {1}
//*---------------------------------------------------------------------        
//DELOLD   EXEC PGM=IEFBR14                                         
//UNLE01P  DD DSN={2},
//            DISP=(MOD,DELETE,DELETE),UNIT=SYSDA,SPACE=(1,(0),RLSE)
//*---------------------------------------------------------------------        
//STEP010  EXEC PGM=IKJEFT01                                                    
//SYSTSIN   DD DSN=ODMLD.PRD.UTIL(DSNX),DISP=SHR                                
//          DD DSN=ODMLD.PRD.UTIL(DSNTEP2),DISP=SHR                             
//SYSTSPRT  DD SYSOUT=*                                                         
//SYSOUT    DD SYSOUT=*                                                         
//*SYSPRINT  DD SYSOUT=*                                                         
//SYSPRINT  DD DSN={2},  
//             DISP=(NEW,CATLG,CATLG),                 
//             UNIT=(SYSDA,5),SPACE=(CYL,(5,5),RLSE)   
//SYSABOUT  DD SYSOUT=*                                                         
//SYSUDUMP  DD SYSOUT=*                                                         
//SYSIN   DD *                            
{3}
/*
//
'''
jcl = jcl_template.format(user, rtc_num, result_file_name, sql)
print( jcl)

jcl_file_name = 'J{}.jcl'.format(rtc_num)
with open(jcl_file_name, 'w') as f:
    f.write(jcl)
print( 'the file {} is created in local..'.format(jcl_file_name))

### STEP 8 Copy the JCL to ODMLD.PRD.RUN(ODMSUBRJ)
  **Attention:**     
  **__If your user id does not have the privilege to write to member ODMSUBRJ, then please ask Hans to manually upload the JCL or SQL statement__**

In [ ]:
from common_func import odm_ftp
with odm_ftp.odm_ftp_conn('put') as odm_put_file:
    server_file_odmsubrj = 'ODMLD.PRD.RUN(ODMSUBRJ)'
    odm_put_file(fm=jcl_file_name, to= server_file_odmsubrj)
    print( 'file {} is uploade in server side...'.format(server_file_odmsubrj))


### STEP 9 Trigger the ODM911 OPC application 

you can trigger the application by yourself or request OPC support team to excute

### STEP 10 Download the dry run result from the MF server 

In [ ]:
stage = 'dry'
with odm_ftp.odm_ftp_conn('get') as odm_get_file:
    print(f'result file name is {result_file_name}')
    file_name = result_file_name.split('.')
    local_file_name = '_'.join([stage, *file_name]) + '.txt'
    print('local file name is : {}'.format(local_file_name))
    odm_get_file(fm=result_file_name, to= local_file_name )
    print( 'file is downloaded to local machine, file name is {}. Please post to jira task as evidence'.format(local_file_name))

text = open(local_file_name, 'r').read()
print(text)

### STEP 11 post the dry run result onto jira task, leave comments and request BPO for approval

### STEP 12 Once approveal from BPO is received, please uploade the JCL again, and perform 'real' run
+ Run from the step 5, 6, 7, 8 again. (to upload the sql/jcl to server) _If you want to dry run again, you can trigger OPC ODMP911 again_
+ in PCOM, edit the member ODMLD.PRD.RUN(ODMSUBRJ), just comment out the step 4 in the SQL (put `-- ` in front of `ROLLBACK` statement
+ Tigger the OPC ODMP911 again, and comfirm the job complete successfully


### STEP 13, Download the result from server to you local machine and then post into jira task, and close the jira task


In [ ]:
stage = 'real'
with odm_ftp.odm_ftp_conn('get') as odm_get_file:
    print(f'result file name is {result_file_name}')
    file_name = result_file_name.split('.')
    local_file_name = '_'.join([stage, *file_name]) + '.txt'
    print('local file name is : {}'.format(local_file_name))
    odm_get_file(fm=result_file_name, to= local_file_name )
    print( 'file is downloaded to local machine, file name is {}. Please post to jira task as evidence'.format(local_file_name))
text = open(local_file_name, 'r').read()
print(text)

### STEP 14, Go to BOX file, populate the close date and change the status for those ticket to be 'completed'